In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from datetime import datetime


In [2]:
driver = webdriver.Chrome()
driver.get("https://supergt.net/pages")

# Wait for dynamic content to load if needed

sched_soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = sched_soup.find_all('tr', {'class':'round_box'})

driver.quit()


In [3]:
# len(schedule_details)

In [4]:
days = [] 
months = []
titles = [] 
descs = []
circuits = []
time = []
start_hours = []
start_minutes = []
end_hours = []
end_minutes = []

year = datetime.now().year

round_counter = 1


month_mapping = {
        'Jan': 1,
        'JAN': 1,
        'January': 1,
        'Feb': 2,
        'FEB': 2,
        'February': 2,
        'Mar': 3,
        'March': 3,
        'Apr': 4,
        'April': 4,
        'May': 5,
        'Jun': 6,
        'June': 6,
        'Jul': 7,
        'July': 7,
        'Aug': 8,
        'August': 8,
        'Sep': 9,
        'September': 9,
        'Oct': 10,
        'October': 10,
        'Nov': 11,
        'November': 11,
        'Dec': 12,
        'DEC': 12,
        'December': 12
    }


lengths = ['300km', '3 Hours', '3 Hours', '350km', '350km', '300km', '3 Hours', '300km']

In [5]:
##Example building info lists

# for i in schedule_details:
#     temp_circuit = i.find('div', {'class': 'race-circuit'}).text
#     circuits.append(temp_circuit)
    
#     temp_date = i.find('div', {'class':"race-date"}).text
#     temp_date = temp_date.split(' ')
#     days.append(int(temp_date[0]))
    
#     temp_month = temp_date[1]
#     converted_month = month_mapping[temp_month]
#     months.append(converted_month)
    
#     round_counter = 1
#     temp_title = 'AsLMS Round' + ' ' + str(round_counter)
#     round_counter += 1
#     titles.append(temp_title)

In [6]:
# <tr class="round_box" id="2024_Round1">
#  <td><img alt="next" height="20" src="/img/icons/icon_next.png" width="20"/></td>
#  <td class="date">4/13-14</td>
#  <td>Round1 OKAYAMA</td>
#  </tr>

for i, length in zip(schedule_details, lengths):
    
    temp_date = i.find('td', {'class':'date'}).text.split('-')
    temp_day = temp_date[-1]
    if '/' in temp_day:
        day = int(temp_day.split('/')[-1])
        month = int(temp_day.split('/')[0])
    else:
        day = int(temp_day)
        month = int(temp_date[0].split('/')[0])
    days.append(day)
    months.append(month)


    temp_title = 'Super GT Round' + ' ' + str(round_counter)
    round_counter += 1
    titles.append(temp_title)
    
    name = i.find_all('td')[-1].text
    desc = name[:5] + ' ' + name[5:] + ' ' + length
    descs.append(desc)

    start_hour = 11
    start_hours.append(start_hour)
        
    if length == '3 Hours':
        end_hour = start_hour + 3
        end_hours.append(end_hour)
    
    else:
        end_hour = start_hour + 4
        end_hours.append(end_hour)
        
    

In [7]:
descs

['Round 1 OKAYAMA 300km',
 'Round 2 FUJI 3 Hours',
 'Round 3 SUZUKA 3 Hours',
 'Round 4 FUJI 350km',
 'Round 5 SUZUKA 350km',
 'Round 6 SUGO 300km',
 'Round 7 AUTOPOLIS 3 Hours',
 'Round 8 MOTEGI 300km']

In [8]:
titles

['Super GT Round 1',
 'Super GT Round 2',
 'Super GT Round 3',
 'Super GT Round 4',
 'Super GT Round 5',
 'Super GT Round 6',
 'Super GT Round 7',
 'Super GT Round 8']

In [9]:
start_hours

[11, 11, 11, 11, 11, 11, 11, 11]

In [10]:
end_hours

[15, 14, 14, 15, 15, 15, 14, 15]

In [12]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [13]:
Super_GT = Calendar()
Super_GT.add('prodid', '-//My calendar product//example.com//')
Super_GT.add('version', '2.0')
uid = 110

In [14]:
 
for title, desc, day, month, s_h, e_h in zip(titles, descs, days, months, start_hours, end_hours):
    ievent = Event()
    ievent.add('summary', title) #Title of the event
    ievent.add('description', desc)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(year, month, day, s_h, 0, 0, tzinfo=pytz.timezone('Asia/Tokyo')))
    ievent.add('dtend', datetime(year, month, day, e_h, 0, 0, tzinfo=pytz.timezone('Asia/Tokyo')))
    ievent.add('dtstamp', datetime(year, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
#     ievent.add('location', circuit)
    ievent.add('uid', uid)
    uid += 1
    Super_GT.add_component(ievent)

In [15]:
Super_GT

VCALENDAR({'PRODID': vText('b'-//My calendar product//example.com//''), 'VERSION': vText('b'2.0'')}, VEVENT({'SUMMARY': vText('b'Super GT Round 1''), 'DESCRIPTION': vText('b'Round 1 OKAYAMA 300km''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000001802076A880>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000001802076A1F0>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000001802076A8B0>, 'UID': vText('b'110'')}), VEVENT({'SUMMARY': vText('b'Super GT Round 2''), 'DESCRIPTION': vText('b'Round 2 FUJI 3 Hours''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x0000018020ADA670>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x0000018020AED1C0>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x0000018020AEDB50>, 'UID': vText('b'111'')}), VEVENT({'SUMMARY': vText('b'Super GT Round 3''), 'DESCRIPTION': vText('b'Round 3 SUZUKA 3 Hours''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x0000018020AE5FA0>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x0000018020AE5700>, 'DTSTAMP': <

In [ ]:
br

In [16]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'Super_GT.ics'), 'wb')
f.write(Super_GT.to_ical())
f.close()

ics file will be generated at  C:\Users\chris\Documents\Calendars/
